# Notes

In [3]:
%matplotlib inline
import sys
sys.path.append("..") #Adds the module to path
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
!git clone https://github.com/gussjos/DeepTrack-2.0

fatal: destination path 'DeepTrack-2.0' already exists and is not an empty directory.


In [4]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
#physical_devices=tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0],True)
#tf.config.gpu.set_per_process_memory_fraction(0.75)
#tf.config.gpu.set_per_process_memory_growth(True)
import numpy as np
import scipy.io as IO
import matplotlib.pyplot as plt
from tensorflow import keras
import scipy.io as IO
import os
K=keras.backend
sys.path.insert(0,'DeepTrack-2.0/')
import deeptrack as dt
# Font parameters
sz = 14
plt.rc('font', size=sz)          # controls default text sizes
plt.rc('axes', titlesize=sz)     # fontsize of the axes title
plt.rc('axes', labelsize=sz)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=sz)    # fontsize of the tick labels
plt.rc('ytick', labelsize=sz)    # fontsize of the tick labels
plt.rc('legend', fontsize=sz)    # legend fontsize
plt.rc('figure', titlesize=sz)  # fontsize of the figure title

## Define network and load weights

In [5]:
# Works with Tensorflow 2.2.0 and Keras 2.3.1
from deeptrack.models import resnetcnn, Convolutional
from deeptrack.losses import unet_features,nd_mean_absolute_error

from keras import models, layers, optimizers, backend, activations
import numpy as np
import tensorflow as tf

def conv_step(layer,dimension):
           layer=layers.Conv2D(dimension,kernel_size=3,activation="relu",padding="same")(layer)
           return layer
        
def _compile(model: models.Model, 
            *,
            loss="mae", 
            optimizer="adam", 
            metrics=[],
            **kwargs):
    ''' Compiles a model.

    Parameters
    ----------
    model : keras.models.Model
        The keras model to interface.
    loss : str or keras loss
        The loss function of the model.
    optimizer : str or keras optimizer
        The optimizer of the model.
    metrics : list, optional
    '''

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model

# Histogram

In [6]:
# ds = (diffs[mask>0.2]).flatten()**2*57
# print(np.mean(ds))

# #ds = ((diffs*mask*np.sum(diffs*mask))**2*57).flatten()
# #ds = ds[ds>0.1]
# H = plt.hist(ds,bins=30)
# #np.sum(ds)**2*57

# Plots to compare output from resnet

In [7]:
from deeptrack.losses import nd_mean_absolute_error
unet_path = '../input/network-weights/unet-1-dec-1415.h5'
unet = tf.keras.models.load_model(unet_path,compile=False)

In [8]:
# j = 9
# preds = resnet.predict(b)

# plt.imshow(np.squeeze(b[j,...]).T)

# # Label mask
# plt.figure()
# Lmask = L[j][...,-1]
# plt.imshow(Lmask.T)#/np.sum(mask))
# plt.colorbar()

# # Mask
# plt.figure()
# mask = np.squeeze(preds[1][j,...,-1]).T
# plt.imshow(mask)#/np.sum(mask))
# plt.colorbar()

# plt.figure()
# diffs = np.squeeze(preds[2][j,...]).T
# plt.imshow(diffs)
# plt.colorbar()

# print('D_label: {}'.format(L[j,0,0,0]**2*57))
# print('D_pred: {}'.format(preds[0][j]**2*57))


# # How diffusion is calculated from the mask
# #np.sum(mask*diffs/np.sum(mask))**2*57
# np.mean(mask*diffs)**2*57
# #print(np.mean(mask*diffs)**2*57)

In [9]:
def plot_predictions(resnet,b,L):
    x = np.linspace(0,1)
    plt.plot(x,x)
    
    D_true = L[:,0,0,0]
    D_pred = resnet.predict(b)[0]
    #print(D_pred.shape)
    #print(D_true.shape)
    plt.scatter(D_true,D_pred) 
    
    #print(resnet.predict(b)[1][...,-1].shape)
    
    masks = resnet.predict(b)[1][...,-1]
    
    fig, ax = plt.subplots(1,3,figsize=(10,6))
    
    j = np.random.randint(batch_size)
    cax = ax[0]
    cax.imshow(L[...,1][j].T)
    
    cax = ax[1]
    cax.imshow(np.squeeze(masks[j]).T)
    
    cax = ax[2]
    plt.imshow(np.squeeze(b[j]).T)
    plt.show()

# Predict on entire images

In [10]:
# Predict on all available data


def predict_on_testdata(resnet, unet, meas_indices, Ds, iOCs, DX, print_outputs):
    # DX: channel_length/2
    
    meas_i1 = meas_indices[0]
    meas_i2 = meas_indices[1]

    def reset_estimates():
        estimates = {'D10_iOC0.0001': {'D':[], 'iOC':[]},
                     'D10_iOC0.0002': {'D':[], 'iOC':[]},
                     'D10_iOC0.0005': {'D':[], 'iOC':[]},
                     'D20_iOC0.0001': {'D':[], 'iOC':[]},
                     'D20_iOC0.0002': {'D':[], 'iOC':[]},
                     'D20_iOC0.0005': {'D':[], 'iOC':[]},
                     'D50_iOC0.0001': {'D':[], 'iOC':[]},
                     'D50_iOC0.0002': {'D':[], 'iOC':[]},
                     'D50_iOC0.0005': {'D':[], 'iOC':[]},};
        return estimates

    estimates = reset_estimates() # store data in estimates

    # Choose path depending on which images to use
    #img_path = '../input/testdata-17-dec-new/'
    img_path = '../input/test-data-no-bgstd-7-jan/'
    std_path = '../input/noise-std-23-dec-2/'

    convert_to_real_units = 1
    images = os.listdir(img_path)

    for iOC in iOCs:
        for D in Ds:
            D_str = 'D' + str(D)
            iOC_str = 'iOC' + str(iOC)
            combination = D_str + '_' + iOC_str
            if print_outputs:
                print(f'Current combination: {iOC_str} & {D_str}')

            # Pick out all measurements for current comb. of D, iOC
            files = [file for file in images if iOC_str in file]
            files = [file for file in files if D_str in file]
            
            for j, file in enumerate(files[meas_i1:meas_i2]):  
                Bsm = np.load(img_path + file, allow_pickle=True)
                
                # Normalize
                #Bsm /= np.max(np.max(Bsm))
                
                Bsm_d = Bsm.T
                Bsm_r = Bsm_d.reshape(1,Bsm_d.shape[0],Bsm_d.shape[1],1) # reshape for input to network
                #Bsm_r = Bsm_r[:1,11:-11,:8192+1024,:1]
                Bsm_r = Bsm_r[:1,11:-11,256:8192+1024-256,:1]
                
                #std = np.load(std_path + file)
                
                #print('a:',np.mean(a))
                #print('b:',np.mean(b))
                
                #print(Bsm_r.shape)
                #segm = unet.predict(Bsm_r.T)
                #segm[segm<0] = 0
                #print(Bsm_r.shape)
                iOC_val, mask = resnet.predict(Bsm_r.T)
                
                #a = std
                iOC_val = iOC_val[0][0]#*a#*1000#*1000#*np.mean(a) #**2*57/256**2 * DX**2
                estimates[combination]['D'].append(iOC_val)
                #estimates[combination]['iOC'].append(iOC_val)
                
                if print_outputs:
                    print(f'file {j+1}/{len(files)} done')
                    print(f'\t iOC_val = {"{:.4f}".format(iOC_val)}')
            if print_outputs:
                print()
            
    return estimates

# iOCs = np.array([1,2,5])*1e-4
# Ds = np.array([10,20,50])
# meas_indices = [0,5]
# print_outputs = 1


#estimates = predict_on_testdata(resnet, unet, meas_indices, Ds, iOCs, DX, print_outputs)
# avg_estimates = calculate_estimates(estimates, iOCs, Ds, print_outputs)

# plot_test_predictions(avg_estimates)        

In [11]:
# Save estimates
import pandas as pd
def calculate_estimates(estimates, iOCs, Ds, print_outputs):
    np.save('estimates.npy',estimates)

    A = np.load('estimates.npy',allow_pickle='TRUE').item()
    avg_estimates = {'D10_iOC0.0001': {'D':[],   'std_D':[],  },
                     'D10_iOC0.0002': {'D':[],   'std_D':[],  },
                     'D10_iOC0.0005': {'D':[],   'std_D':[],  },
                     'D20_iOC0.0001': {'D':[],   'std_D':[],  },
                     'D20_iOC0.0002': {'D':[],   'std_D':[],  },
                     'D20_iOC0.0005': {'D':[],   'std_D':[],  },
                     'D50_iOC0.0001': {'D':[],   'std_D':[],  },
                     'D50_iOC0.0002': {'D':[],   'std_D':[],  },
                     'D50_iOC0.0005': {'D':[],   'std_D':[],  },}
    # Average over combinations
    nbr_segments = 1
    for iOC in iOCs:
        for D in Ds:
            iOC_str = 'iOC' + str(iOC)
            D_str = 'D' + str(D)
            combination = D_str + '_' + iOC_str

            D_vals = np.array(A[combination]['D'])
            #print(D_vals)
            #print(np.mean(D_vals))
            avg_estimates[combination]['D'] = np.mean(D_vals)
            avg_estimates[combination]['std_D'] = np.std(D_vals)

    est_df = pd.DataFrame(avg_estimates)
    if print_outputs:
        display(est_df)
    return avg_estimates


In [12]:
# Extract averages per simulated iOC #2

def plot_test_predictions(avg_estimates,plot_Ds, I_scaling_factor):
    iOC_arr_D10 = []
    iOC_arr_D20 = []
    iOC_arr_D50 = []
    
    iOC_std_D10 = []
    iOC_std_D20 = []
    iOC_std_D50 = []

    D50 = np.array(3*[50])
    D20 = np.array(3*[20])
    D10 = np.array(3*[10])

    for j in [1,2,5]:
        est_str = f'D10_iOC0.000{j}'
        est = avg_estimates[est_str]
        
        iOC_arr_D10.append(est['D']*I_scaling_factor)
        iOC_std_D10.append(est['std_D']*I_scaling_factor)

    for j in [1,2,5]:
        est_str = f'D20_iOC0.000{j}'
        est = avg_estimates[est_str]
        
        iOC_arr_D20.append(est['D']*I_scaling_factor)
        iOC_std_D20.append(est['std_D']*I_scaling_factor)


    for j in [1,2,5]:
        est_str = f'D50_iOC0.000{j}'
        est = avg_estimates[est_str]
        
        iOC_arr_D50.append(est['D']*I_scaling_factor)
        iOC_std_D50.append(est['std_D']*I_scaling_factor)


    plt.figure(figsize=(5,5))

    elw = 1.5
    if plot_Ds[2]:
        plt.scatter(iOC_arr_D50, D50, label='D=50', color='tab:blue')
        plt.errorbar(iOC_arr_D50, D50, xerr = iOC_std_D50, fmt='.',
                        capsize=5, elinewidth=elw, color = 'tab:blue')
    if plot_Ds[1]:
        plt.scatter(iOC_arr_D20, D20, label='D=20', color='tab:orange')
        plt.errorbar(iOC_arr_D20, D20, xerr = iOC_std_D20, fmt='.',
                        capsize=5, elinewidth=elw, color = 'tab:orange')
    if plot_Ds[0]:
        plt.scatter(iOC_arr_D10, D10, label='D=10', color='tab:green')
        plt.errorbar(iOC_arr_D10, D10, xerr = iOC_std_D10, fmt='.',
                        capsize=5, elinewidth=elw, color = 'tab:green')



    # Plot expected D-vals
    #plt.axvline(1e-4,linewidth = 0.65,linestyle='--',color='black')
    #plt.axvline(2e-4,linewidth = 0.65,linestyle='--',color='black')
    #plt.axvline(5e-4,linewidth = 0.65,linestyle='--',color='black')


    #plt.text(np.min(D_arr_iOC5), np.mean(iOC_arr_iOC5)+h,f'{"{:.2f}".format(c5)}x iOC 1e-4')
    #plt.text(np.min(D_arr_iOC5), np.mean(iOC_arr_iOC2)+h,f'{"{:.2f}".format(c2)}x iOC 1e-4')
    #plt.text(np.min(D_arr_iOC5), np.mean(iOC_arr_iOC1)+h,f'iOC 1e-4')
    
    #plt.xlim(0,6e-4)
    plt.ylim(0,60)

    plt.xlabel('iOC (1e-4)', fontsize=14)
    plt.ylabel(r'$D [\mu$m$^2$/s]', fontsize=14)
    plt.title('Mean predictions over all measurements',fontsize=14)
    plt.legend(fontsize=12)
    plt.show()


## Comment: Run all above Train for plots to work in training loop

In [ ]:
unet = None
resnet.load_weights('../input/resnet-11-jan-int/intensities_11jan.h5')

meas_indices = [0,10]
print_outputs = 1
length = 512
DX = length/2
D_arr = [10,20,50]
estimates = predict_on_testdata(resnet, unet, meas_indices, D_arr, iOCs, DX, print_outputs)
avg_estimates = calculate_estimates(estimates, iOCs, D_arr, print_outputs)

plot_Ds = [1,1,1]
I_scaling_factor = 1
plot_test_predictions(avg_estimates,plot_Ds,I_scaling_factor) 

In [ ]:
print('D10')
print(avg_estimates['D10_iOC0.0002']['D'] - avg_estimates['D10_iOC0.0001']['D'])
print(avg_estimates['D10_iOC0.0005']['D'] - avg_estimates['D10_iOC0.0001']['D'])
print('D20')
print(avg_estimates['D20_iOC0.0002']['D'] - avg_estimates['D20_iOC0.0001']['D'])
print(avg_estimates['D20_iOC0.0005']['D'] - avg_estimates['D20_iOC0.0001']['D'])
print('D50')
print(avg_estimates['D50_iOC0.0002']['D'] - avg_estimates['D50_iOC0.0001']['D'])
print(avg_estimates['D50_iOC0.0005']['D'] - avg_estimates['D50_iOC0.0001']['D'])

print(avg_estimates['D50_iOC0.0005']['D'])
print(avg_estimates['D50_iOC0.0002']['D'])
print(avg_estimates['D50_iOC0.0001']['D'])

In [ ]:
plot_Ds = [1,1,1]
D_arr = np.array([10,20,50])
I_scaling_factor = 1
plot_test_predictions(avg_estimates,plot_Ds,I_scaling_factor) 

# Plot testdata for visualization

In [ ]:
j = 51
path = '../input/test-data-no-bgstd-7-jan/'
#path = '../input/testdata-17-dec-new/'
files = os.listdir(path)
filename = files[j]

filename = '24-08-20_16-47-20_D20_iOC0.0001_M.mat.npy'
file = path + filename
print(file)
A = np.load(file)

# Plot testdata
plt.figure(figsize=(16,2))
i1 = 1024
plt.imshow(A.T[:,i1:i1+1024],aspect = 'auto')
plt.colorbar()

# Plot segmentation of testdata 
N = int((150-128)/2)
B = A[:8192,N:-N]
B = np.reshape(B,(1,8192,128,1))
P = np.squeeze(unet.predict(B)).T
plt.figure(figsize=(16,2))
plt.imshow(P[:,i1:i1+1024],aspect = 'auto')
plt.colorbar()

In [ ]:
j = 1
plt.imshow(b[j,...,0].T)
plt.colorbar()

p = resnet.predict(b)[0][j]

print('pred:',p)
print('label:',L[j,0,0,0])

In [14]:
for j in range(0,8):
    im = b[j,...,0]
    A = np.std(im,axis=0)
    A = skimage.measure.block_reduce(A,(25,), np.mean)
    plt.plot(A)

NameError: name 'b' is not defined

# Define images

In [16]:
I = 0.5
s = 0.04
gauss_integral = s*np.sqrt(2*np.pi)
train_dx = 256
real_dx = 0.03
conv_factor = train_dx*real_dx
I/(gauss_integral*conv_factor)

0.6493201178408736

In [13]:
import deeptrack as dt
from deeptrack.features import Feature
import numpy as np
import skimage.measure

# class set_params(Feature):
    
#     def __init__(self, vel=0, D=0, I=0, s=0, **kwargs):
#         super().__init__(
#             vel=vel, D=D, I=I,s=s, **kwargs
#         )

#     def get(self, image, vel, D, I,s, **kwargs):
#         Int = lambda : 1e-3 * (0.6)#0.4*np.random.rand())
#         Ds = 0.1*np.sqrt(0.05+1.5*np.random.rand()) # Diffusions are defined in get_diffusion()!   lambda: 0.10*(0.05+1.5*np.random.rand())#np.sqrt((0.05 + np.random.rand()))
#         st = lambda: 0.04 + 0.01*np.random.rand()
#         image.append({"D": 10*D})
#         image.append({"I": s*I*np.sqrt(2*np.pi)*256*.03*1000})
#         image.append({"s": s})
        
#         return image
    
class get_diffusion_and_intensity(Feature):
    
    def __init__(self, vel=0, D=0, I=0, s=0, **kwargs):
        super().__init__(
            vel=vel, D=D, I=I,s=s, **kwargs
        )

    def get(self, image, vel, D, I,s, **kwargs):
        LOW = 0.01
        HIGH = 1
        D = (LOW + HIGH*np.random.rand())
        image.append({"D":D})
        
        s = 0.04 + 0.01*np.random.rand()
        image.append({"s":s})
        
        I1 = 0.001
        I2 = 0.5
        I_max = I1+I2
        I = 1e-3 * (I1+I2*np.random.rand())
        image.append({"I_initial":s*I*np.sqrt(2*np.pi)*256*.03*1000/I_max})
        
        return image
    
class get_trajectory(Feature):

    
    def __init__(self, vel=0, D=0.1, I=0.1,s=0.05, **kwargs):
        super().__init__(
            vel=vel, D=D, I=I,s=s, **kwargs
        )

    def get(self, image, vel, D, I,s, **kwargs):
        
        
        D = image.get_property("D")/10
        I = image.get_property("I_initial")/1000
        s = image.get_property("s")
        
        length=image.shape[1]
        times=image.shape[0]
        x=np.linspace(-1,1,length)
        t=np.linspace(-1,1,times)
        X, Y=np.meshgrid(t,x)
        G= lambda a,b,x0,s,x: a*np.exp(-(x-x0)**2/s**2)+b
        f2=lambda a,x0,s,b,x: a*np.exp(-(x-x0)**2/s**2)+b
        x0=0
        x0+=np.cumsum(vel+D*np.random.randn(times))
        v1=np.transpose(I*f2(1,x0,s,0,Y))
        image[...,0]*=(1-v1)
        image[...,1]+=np.transpose(f2(1,x0,0.05,0,Y))
        #image[...,2]+=np.transpose(I*f2(1,x0,s,0,Y))
        
#         try:
#             image.properties["D"]+=10*D#*np.sum(np.transpose(f2(1,x0,.1,0,Y)))
#             image.properties["I"]+=s*I*np.sqrt(2*np.pi)*256*.03*1000
#         except:
#             image.append({"D":10*D,"I":s*I*np.sqrt(2*np.pi)*256*.03*1000})
            
        #imaged_sample[...,0]=v1
        #imaged_sample.properties=self.properties
        return image
    
class gen_noise(Feature):
    
    
    def __init__(self, noise_lev=0, dX=0, dA=0,biglam=0,bgnoiseCval=0,bgnoise=0,bigx0=0, **kwargs):
        super().__init__(
            noise_lev=noise_lev, dX=dX, dA=dA,biglam=biglam,bgnoiseCval=bgnoiseCval,bgnoise=bgnoise,bigx0=bigx0, **kwargs
        )

    def get(self, image, noise_lev, dX, dA,biglam,bgnoiseCval,bgnoise,bigx0, **kwargs):
        #image=np.zeros((256,512,2))
        from scipy.signal import convolve
        
        length=image.shape[1]
        times=image.shape[0]
        x=np.linspace(-1,1,length)
        t=np.linspace(-1,1,times)
        X, Y=np.meshgrid(t,x)
        f2=lambda a,x0,s,b,x: a*np.exp(-(x-x0)**2/s**2)+b
        bgnoise*=np.random.randn(length)

        tempcorr=3*np.random.rand()
        dAmp=dA#*np.random.rand()
        shiftval=dX*np.random.randn()
        dx=0
        dx2=0
        dAmp0=0
        
        bg0=f2(1,bigx0,biglam,0,x)
        image.append({"bg0":bg0})
        
        
        ll=(np.pi-.05)
        for j in range(times):
            #dx=np.sqrt(1-np.exp(-2/tempcorr))*dX*np.random.randn()+np.exp(-1/tempcorr)*dx
            #dx2=np.sqrt(1-np.exp(-2/tempcorr))*dX*np.random.randn()+np.exp(-1/tempcorr)*dx2
            dx=(.7*np.random.randn()+np.sin(ll*j))*dX
            
            bgnoiseC=f2(1,0,bgnoiseCval,dx,x)
            #bgnoiseC2=f2(1,0,bgnoiseCval,dx2+shiftval,x)
            bgnoiseC/=np.sum(bgnoiseC)
            bg=f2(1,bigx0+dx,biglam,0,x)*(1+convolve(bgnoise,bgnoiseC,mode="same"))
            #bg2=f2(1,bigx0+dx2+shiftval,biglam,0,x)+convolve(bgnoise,bgnoiseC2,mode="same")
            #dAmp0=np.sqrt(1-np.exp(-2/tempcorr))*dAmp*np.random.randn()+np.exp(-1/tempcorr)*dAmp0
            dAmp0=dA*np.random.randn()
            bg*=(1+dAmp0)
            image[j,:,0]=bg*(1+noise_lev*np.random.randn(length))+.4*noise_lev*np.random.randn(length)
        
        return image
     

    
class post_process(Feature):
    
    
    def __init__(self, noise_lev=0, dX=0, dA=0, **kwargs):
        super().__init__(
            noise_lev=noise_lev, dX=dX, dA=dA, **kwargs
        )

    def get(self, image, **kwargs):
        from scipy.signal import convolve2d
        length=image.shape[1]
        times=image.shape[0]
        x=np.linspace(-1,1,length)
        t=np.linspace(-1,1,times)
        X, Y=np.meshgrid(t,x)
        G= lambda a,b,x0,s,x: a*np.exp(-(x-x0)**2/s**2)+b
        f2=lambda a,x0,s,b,x: a*np.exp(-(x-x0)**2/s**2)+b                         
        
        
        bg0 = image.get_property("bg0")
        image[:,:,0]/=bg0 # Normalize image by the bare signal
        
        #image[:,:,0]/=np.mean(image[...,0],axis=0)        
        image[:,:,0]-=np.expand_dims(np.mean(image[:,:,0],axis=0),axis=0) # Subtract mean over image

        # Perform same preprocessing as done on experimental images
        ono=np.ones((200,1))
        ono[0:80]=1
        ono[120:]=1
        ono=ono/np.sum(ono)
        image[:,:,0]-=convolve2d(image[:,:,0],ono,mode="same")
        image[:,:,0]-=convolve2d(image[:,:,0],np.transpose(ono),mode="same")
        
        
        
        image[:,:,0]-=np.expand_dims(np.mean(image[:,:,0],axis=0),axis=0)
        image[:,:,0]*=2000
        
        #image[:,:,0] /= np.max(np.max(image[:,:,0]))
        
        
        #a=np.std(image[...,0],axis=0)
        #print(a)
        #image[:,:,0]/=a
        #image.properties["I"] /= 1000*np.mean(a)
        #image.append({"a":a})
        
        
        #image.properties["I"] *= 1000
#         I = image.get_property("I_initial")
#         print(I/np.mean(a)/1000)
#         image.append({"I": I/np.mean(a)})
        
#        I = image.get_property("I_initial")
#         try:
#             image.append({"I": I(a)})
#             #print(I/np.mean(a))
#         except:
#             print('"I" not divided by a')
        #image[...,2]/=a
        #image=np.flip(image,axis=-1)
        
        
        
        return image

    
class input_array(Feature):
    __distributed__ = False
    def get(self,image, **kwargs):
        image=np.zeros((times,length,2))
        return image

        
def batch_function(image):
    img = image[...,:1]
    img = skimage.measure.block_reduce(img,(T_reduction_factor,L_reduction_factor,1),np.mean)
    #GAN_output = unet.predict(np.expand_dims(img,axis=0))
    #mask = skimage.measure.block_reduce(GAN_output, (1,64,16,1), np.mean)
    #img = np.expand_dims(img,axis=0)
    #print('GAN_output[0].shape',GAN_output[0].shape)
    return img #GAN_output[0] #, mask

def label_function(image):
    D = image.get_property("D")
    I = image.get_property("I_initial")
    #I *= 2000
    #a = image.get_property("a")
    #I = image.get_property("I_initial")
    #I /= a*1000
    #print("I:",I)
    
    if np.sum(image[...,1]) < 2:
        D = 0
        I = 0
    t_final = 8
    L_final = 8
    time_reduction = int(image.shape[0]/t_final)
    len_reduction = int(image.shape[1]/L_final)
    
    #std_reduction = int(length/L_final)
    #a = skimage.measure.block_reduce(a,(std_reduction,),np.mean)
    mask = skimage.measure.block_reduce(image[...,1:2], (time_reduction,len_reduction,1), np.max)
    #I = np.mean(mask/a)/1000
    
    #print('mask.shape',mask.shape)

    labels = np.ones((t_final,L_final,2))
    labels[...,0] *= I
    labels[...,1:] = mask
    #labels[...,-1] *= a
    
    return labels

mae = tf.keras.losses.MeanAbsoluteError()

def diffusion_loss(T,P):
    D_true = T[:,0,0,0] # Diffusion
    D_pred = P
    
    return mae(D_true,D_pred) #+ mLoss

def zeroloss(T,P):
    return tf.constant(0)

def mask_loss(ytrue,ypred):    
    
    #T = ytrue[...,-1]
    #P = ypred[...,-1]
    
    T = ytrue[...,1]
    P = ypred[...,-1]
    #T=K.flatten(ytrue[...,1])
    #P=K.flatten(ypred[...,-1])
    
    #T=K.flatten(ytrue)
    #P=K.flatten(ypred)
    loss1=-K.mean(T*K.log(P+1e-3)+(1-T)*K.log(1-P+1e-3))
    #loss2=K.mean(T*K.abs(K.flatten(ytrue[...,0]-ypred[...,0])))+K.mean(T*K.abs(K.flatten(ytrue[...,1]-ypred[...,1])))
    return loss1 #+ loss2

def generate_training_batch(net,image,batch_size):
    
    min_data_size = batch_size
    max_data_size = batch_size+1

    #convnet.load_weights('convnet.h5')
    data_generator=dt.generators.ContinuousGenerator(image,
                                                batch_function=batch_function,
                                                label_function=label_function,
                                                batch_size=batch_size,
                                                min_data_size=min_data_size,
                                                max_data_size=max_data_size)

    with data_generator:
        net.fit(data_generator, epochs=0, steps_per_epoch=1)
        
    b,L = data_generator[0] 
    return b,L


def plot_predictions(resnet,b,L):
    x = np.linspace(0,1)
    plt.plot(x,x)
    
    D_true = L[:,0,0,0]
    D_pred = resnet.predict(b)[0]
    #print(D_pred.shape)
    #print(D_true.shape)
    plt.scatter(D_true,D_pred) 
    
    #print(resnet.predict(b)[1][...,-1].shape)
    
    masks = resnet.predict(b)[1][...,-1]
    
    
    for img in range(0,2):
        fig, ax = plt.subplots(1,2)
        j = np.random.randint(batch_size)
        cax = ax[0]
        cax.imshow(L[...,1][j].T)

        cax = ax[1]
        cax.imshow(np.squeeze(masks[j]).T)
        plt.show()

optimizer = tf.keras.optimizers.Adam




# TRAIN

In [ ]:
### Params to change below ###
TRAIN = 1
reset_resnet = 0
load_resnet = 0
GEN_NEW_VAL_DATA = 1 # Good thing to generate new val-data when changing parameters or when running for the first time          
val_bsize = 16
DEBUG = 0

#resnet_to_load = '../input/network-weights/resnet_1024x512_low_contrast.h5'
#resnet_to_load = './resnet_intensity_dec16.h5'
#resnet_to_load = '../input/resnet-intensity-16-dec/resnet_intensity_dec16.h5'
resnet_to_load = '../input/network-weights/resnet_intensity_dec-15.h5'
resnet_to_load = '../input/intensities-11jan-nicelyseparatedh5/intensities_11-jan_nicely-separated.h5'

from deeptrack.models import resnetcnn
if reset_resnet:
    resnet=resnetcnn(input_shape=(None, None, 1),
            conv_layers_dimensions=(16, 32, 64, 128, 256), # sets downsampling size
            upsample_layers_dimensions=(64, 128),
            base_conv_layers_dimensions=(128, 128),
            output_conv_layers_dimensions=(16, 16),
            dropout=(),#0.01,
            pooldim=2,
            steps_per_pooling=1,
            number_of_outputs=1,
            output_activation=None,
            loss="mae",
            layer_function=None,
            BatchNormalization=False,
            conv_step=None)


# Choose loss functions when compiling
resnet.compile(optimizer(lr=0.0001, amsgrad=True), loss=[diffusion_loss,mask_loss])
    
# Define image dimensions
length = 1024
L_reduction_factor = 4
reduced_length = int(length/L_reduction_factor)

times = 256
T_reduction_factor = 1
reduced_times = int(times/T_reduction_factor)

nbrParticles = lambda: np.clip(np.random.randint(5),0,1)

### Defined with images
Int = 0
Ds = 0 
st = 0
###

image=dt.FlipLR(dt.FlipUD(input_array() + get_diffusion_and_intensity() + #set_params(D=Ds, I=Int, s=st) + #get_diffusion(D=Ds) + get_intensity(I=Int,s=st) + 
                          gen_noise(dX=.00001+.00003*np.random.rand(),
                                                  dA=0,
                                                  noise_lev=.0001,
                                                  #noise_lev = 0.0001*(0.75+0.75*np.random.rand()),
                                                  biglam=0.6+.4*np.random.rand(),
                                                  bgnoiseCval=0.03+.02*np.random.rand(),
                                                  #bgnoiseCval=0.03+.05*np.random.rand(),
                                                  #bgnoise=.08+.14*np.random.rand(),
                                                  bgnoise=.08+.04*np.random.rand(),
                                                  bigx0=lambda: .1*np.random.randn())
                                      + get_trajectory(I=Int,s=st)**nbrParticles
                                      + post_process()))

batch_size = 16
min_data_size = 16
max_data_size = 128

if DEBUG:
    batch_size = 1
    min_data_size = 1
    max_data_size = 2

data_generator = dt.generators.ContinuousGenerator(image,
                                    batch_function=batch_function,
                                    label_function=label_function,
                                    batch_size=batch_size,
                                    min_data_size=min_data_size,
                                    max_data_size=max_data_size)                
if GEN_NEW_VAL_DATA:
    print('Generating validation data:')
    b,L = generate_training_batch(resnet,image,batch_size=val_bsize)

    
if load_resnet:
    resnet.load_weights(resnet_to_load)
    
# Combinations to use in phase plot
iOCs = np.array([1,2,5])*1e-4
#iOCs = np.array([5])*1e-4
plot_Ds = [1,1,1]
#D_arr = np.array([10,20,50])
D_arr = np.array([10,20,50])

make_phase_plot = 1
phase_plot_freq = 5
full_phase_plot_freq = 20

make_val_plot = 1
val_plot_freq = 5

print_outputs = 0
inner_epochs = 10
numberLoops = 1000

epochs = 0
total_nr_epochs = 0
print('Generating training data:')
if TRAIN:
    with data_generator:
        for epochs in range(numberLoops):
            resnet.fit(data_generator, epochs=inner_epochs, steps_per_epoch=4)
            epochs += inner_epochs
            print('{} epochs performed.'.format(epochs))
            
            # Plot predictions of validation data
            if make_val_plot and epochs % val_plot_freq == 0:
                plot_predictions(resnet,b,L)

                
            # Make predictions on ALL test data
            if make_phase_plot and epochs % phase_plot_freq == 0: 
                
                if epochs % full_phase_plot_freq == 0:
                    meas_indices = [0,10]
                
                else:
                    R = np.random.randint(0,7)
                    meas_indices = [R,R+2]
                DX = length/2
                unet = None
                estimates = predict_on_testdata(resnet, unet, meas_indices, D_arr, iOCs, DX, print_outputs)
                avg_estimates = calculate_estimates(estimates, iOCs, D_arr, print_outputs)
                
                print('D10')
                print(avg_estimates['D10_iOC0.0002']['D'] - avg_estimates['D10_iOC0.0001']['D'])
                print(avg_estimates['D10_iOC0.0005']['D'] - avg_estimates['D10_iOC0.0001']['D'])
                print('D20')
                print(avg_estimates['D20_iOC0.0002']['D'] - avg_estimates['D20_iOC0.0001']['D'])
                print(avg_estimates['D20_iOC0.0005']['D'] - avg_estimates['D20_iOC0.0001']['D'])
                print('D50')
                print(avg_estimates['D50_iOC0.0002']['D'] - avg_estimates['D50_iOC0.0001']['D'])
                print(avg_estimates['D50_iOC0.0005']['D'] - avg_estimates['D50_iOC0.0001']['D'])

                
                I_scaling_factor = 1
                
                plot_test_predictions(avg_estimates,plot_Ds, I_scaling_factor)   
                
                            

#             # Plot predictions of one randomly selected test measurement
#             if make_test_plots and epochs % 5 == 0 and not (epochs % 20 == 0):
#                 R = np.random.randint(9)
#                 meas_indices = [R,R+1]
#                 DX = length/2
#                 estimates = predict_on_testdata(resnet, unet, meas_indices, D_arr, iOCs, DX, print_outputs)
#                 avg_estimates = calculate_estimates(estimates, iOCs, D_arr, print_outputs)

#                 plot_test_predictions(avg_estimates)   

#    return b,L
#b,L = generate_training_batch(image,1)

# plt.figure(figsize=(10,2))
# plt.imshow(im[...,0].T,aspect='auto')
# plt.colorbar()
    
# plt.figure(figsize=(10,2))
# plt.imshow(im[...,1].T,aspect='auto')
# plt.colorbar()

# print(im.get_property("D"))
# print(im.get_property("I"))

resolve_image = 0
if resolve_image:
    im=image.update().resolve()
    DX = length/2
    im=image.update().resolve()
    print('D: {:.2f}'.format(im.get_property("D")**2*57/(256**2)*DX**2))
    print('I: {:.2f}e-4'.format(im.get_property("I")*10))
    print('s: {:.2f}'.format(im.get_property("s")))



In [ ]:
resnet.save('intensities_11-jan_nicely-separated.h5')

In [ ]:
path = '../input/test-data-no-bgstd-7-jan/24-08-20_16-06-15_D10_iOC0.0001_M.mat.npy'
A = np.load(path)
A = A[256:512]
plt.plot(np.std(A,axis=0))
plt.figure()
plt.imshow(np.squeeze(A).T,aspect='auto')
plt.colorbar()

In [ ]:
tot_img = np.zeros_like(b[0,...,0])
N = 16
for j in range(0,N):
    tot_img += b[j,...,0]
tot_img /= N
plt.imshow(tot_img)
plt.colorbar()

In [ ]:
j = 22
plt.plot(np.std(b[j,...,0],axis=0))
plt.figure()
img = b[j,...]
img /= np.max(np.max(img))
plt.imshow(np.squeeze(img).T,aspect='auto')
plt.colorbar()
print(L[j,0,0,0])

In [ ]:
for j in range(10):
    plt.figure(figsize=(8,2))
    plt.imshow(b[j,...,0].T,aspect='auto')
    I = L[j,0,0,0]
    plt.title('I:{:.2f}'.format(I))
    plt.colorbar()

In [ ]:
resnet.save('resnet_intensity_jan7.h5')

In [ ]:
L[0,:,:,0]

In [ ]:
L[0,0,0,0]

In [ ]:
j = 1
im = b[...,j]

plt.figure(figsize=(10,2))
plt.imshow(im[...,0].T,aspect='auto')
plt.colorbar()
    
plt.figure(figsize=(10,2))
plt.imshow(im[...,1].T,aspect='auto')
plt.colorbar()

In [ ]:
j = 2
I = L[j,0,0,0]
print(I/1000)
print(I)

In [ ]:
resnet.save('resnet_intensity_dec-15.h5')

# Predict on all measurements

In [ ]:
meas_indices = [0,10]
print_outputs = 0
DX = length/2
estimates = predict_on_testdata(resnet, unet, meas_indices, D_arr, iOCs, DX, print_outputs)
avg_estimates = calculate_estimates(estimates, iOCs, D_arr, print_outputs)

plot_Ds = [1,1,1]
plot_test_predictions(avg_estimates,plot_Ds) 

# Loop over images to make predictions (not used right now)

In [ ]:
Dtot = 0
file = '../input/testdata-1-dec/24-08-20_16-06-15_D50_iOC0.0005_M.mat.npy'
file = '../input/testdata-1-dec/24-08-20_17-17-45_D0_iOC0.0001_M.mat.npy'
Bsm_full = np.load(file)
L = Bsm_full.shape[0]
N = int(L/128)
for j in range(0,int(N/3)):
    Bsm = np.copy(Bsm_full[j*128:j*128+128,:128])
    Bsm = np.flip(Bsm,axis = 0)
    img = np.reshape(Bsm,(1,128,128,1))
    segmentation = unet.predict(img)
    pred = resnet.predict(segmentation)
    DP = pred[0][0][0]
    DPreal = (DP**2*57*2)
    Dtot+=DPreal
    #print(DPreal)
print('Avg:',Dtot/N)